In [2]:
from torch import nn
import sys
sys.path.insert(0,'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
#sys.path.insert(0,'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/')
import numpy as np
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from torchvision import transforms
from torch.autograd import Variable
import os
import pandas as pd
import seaborn as sns
from dataloaders import get_train_transforms, get_val_transforms, get_triplet_dataloader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from sklearn.metrics import accuracy_score, f1_score , precision_score , recall_score
from sklearn.manifold import TSNE
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
path_data = 'C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
#path_data = 'f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/src/dataset'
#device = torch.cuda.is_available()
def generate_embeddings(data_loader, model):
    with torch.no_grad():
        model.eval()
        all_embeddings = []
        all_labels = []
        
        for batch_imgs, batch_labels in data_loader:
            if torch.cuda.is_available():
                batch_imgs = batch_imgs.cuda()
            
            B, C, H, W = batch_imgs.shape
            patch_size = 64  # افزایش سایز پچ برای جلوگیری از خطای kernel
            stride = H // 14
            
            batch_embeddings = torch.zeros(B, 14, 14, model.get_embedding(batch_imgs[:1]).shape[-1]).to(batch_imgs.device)
            
            # استخراج پچ‌ها و محاسبه embeddings با حفظ موقعیت مکانی
            for i in range(14):
                start_h = i * stride
                end_h = min(start_h + patch_size, H)
                if end_h - start_h < patch_size:
                    start_h = max(0, end_h - patch_size)
                
                for j in range(14):
                    start_w = j * stride
                    end_w = min(start_w + patch_size, W)
                    if end_w - start_w < patch_size:
                        start_w = max(0, end_w - patch_size)
                    
                    patch = batch_imgs[:, :, start_h:end_h, start_w:end_w]
                    if patch.shape[2:] != (patch_size, patch_size):
                        patch = F.interpolate(patch, size=(patch_size, patch_size), mode='bilinear', align_corners=True)
                    
                    batch_embeddings[:, i, j] = model.get_embedding(patch)
            
            all_embeddings.append(batch_embeddings.cpu().numpy())
            all_labels.append(batch_labels.numpy())
            
        return np.concatenate(all_embeddings), np.concatenate(all_labels)

class RefinedViT(nn.Module):
    def __init__(self, original_vit_model, embedding_dim, num_classes):
        super().__init__()
        self.num_patches = 196
        self.embed_dim = embedding_dim
        
        # اضافه کردن تعداد هدها به صورت صریح
        self.num_attention_heads = original_vit_model.config.num_attention_heads
        
        self.pos_embed = nn.Parameter(torch.zeros(1, self.num_patches + 1, self.embed_dim))
        self.cls_token = nn.Parameter(torch.zeros(1, 1, self.embed_dim))
        
        # حفظ کامل encoder با مالتی‌هد اتنشن
        self.encoder = original_vit_model.encoder
        self.layernorm = nn.LayerNorm(self.embed_dim)
        self.fc = nn.Linear(self.embed_dim, num_classes)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B = x.shape[0]
        x = x.view(B, 196, -1)
        
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        
        # استفاده از مالتی‌هد اتنشن در encoder
        x = self.encoder(x)
        x = self.layernorm(x[:, 0])
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

 

# Initialize models
# if device:
#patch_extractor = PatchExtractor(patch_size=14).cuda()
#patch_extractor = PatchExtractor(patch_size=14)   
# Load your data
train_data = torchvision.datasets.ImageFolder(root=path_data + '/train/', transform=get_val_transforms())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'  
siamese_model = torch.load("C:/Users/Mey/Documents/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_model_EmbeddingGEnerator.h5",map_location=torch.device(device))
#siamese_model = torch.load("f:/Meysam-Khodarahi/PlantDiseaseDiagnosisFewShotLearning/siamese_triplet_net/siamese_model_EmbeddingGEnerator.h5",map_location=torch.device('cuda'))
siamese_model.eval()
# Generate embeddings
 

 
 

# بارگذاری مدل ViT از Hugging Face
vit_model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
vit_model.classifier = torch.nn.Linear(vit_model.config.hidden_size, 15)  # تعداد کلاس ها

embedding_dim = vit_model.config.hidden_size  # ابعاد embedding
 


train_data = torchvision.datasets.ImageFolder(root=path_data + '/train/', transform=get_val_transforms())
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32)

test_data = torchvision.datasets.ImageFolder(root=path_data + '/test/', transform=get_val_transforms())
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32)

# استخراج ویژگی ها از داده های آموزش
train_embeddings, train_labels = generate_embeddings(train_loader, siamese_model )
# استخراج ویژگی ها از داده های تست
test_embeddings, test_labels = generate_embeddings(test_loader, siamese_model )
 
 

def train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels, 
                      num_epochs=100, batch_size=32, learning_rate=1e-4):
    print("train_and_validate")
    # تبدیل به تنسور
    X_train = torch.tensor(train_embeddings, dtype=torch.float32)
    y_train = torch.tensor(train_labels, dtype=torch.long)
    X_val = torch.tensor(val_embeddings, dtype=torch.float32)
    y_val = torch.tensor(val_labels, dtype=torch.long)
    
    # انتقال به GPU
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)
    
    # تنظیمات آموزش
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    
    # آموزش
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        total_train_loss = 0
        num_batches = len(X_train) // batch_size + (1 if len(X_train) % batch_size != 0 else 0)
        
        for i in range(0, len(X_train), batch_size):
            batch_X = X_train[i:i+batch_size]
            batch_y = y_train[i:i+batch_size]
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
            
        avg_train_loss = total_train_loss / num_batches
        train_losses.append(avg_train_loss)
        
        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val)
            val_loss = criterion(val_outputs, y_val).item()
            _, predicted = torch.max(val_outputs.data, 1)
            val_accuracy = (predicted == y_val).float().mean().item()
            
        val_losses.append(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), 'best_modelState_Rvit.pth')
            torch.save(model , 'best_model_Rvit.pth')
        
        scheduler.step()
        
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}]:')
            print(f'Train Loss: {avg_train_loss:.4f}')
            print(f'Val Loss: {val_loss:.4f}')
            print(f'Val Accuracy: {val_accuracy:.4f}')
            print('-' * 50)
    
    # Plot training curves
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.title('Loss Curves')
    plt.legend()
    plt.show()
    
    return model

# Usage:
print("model")
model = RefinedViT(vit_model, embedding_dim, num_classes=15)
trained_model = train_and_validate(model, train_embeddings, train_labels, val_embeddings, val_labels)
 

def visualize_embeddings(train_embeddings, train_labels, val_embeddings, val_labels, test_embeddings, test_labels):
    # Reshape embeddings for t-SNE
    train_flat = train_embeddings.reshape(train_embeddings.shape[0], -1)
    val_flat = val_embeddings.reshape(val_embeddings.shape[0], -1)
    test_flat = test_embeddings.reshape(test_embeddings.shape[0], -1)
    
    # Combine all data for t-SNE
    combined_data = np.vstack([train_flat, val_flat, test_flat])
    combined_labels = np.concatenate([train_labels, val_labels, test_labels])
    
    # Apply t-SNE
    tsne = TSNE(n_components=2, random_state=42)
    embeddings_2d = tsne.fit_transform(combined_data)
    
    # Split back into train/val/test
    n_train = len(train_labels)
    n_val = len(val_labels)
    
    train_2d = embeddings_2d[:n_train]
    val_2d = embeddings_2d[n_train:n_train+n_val]
    test_2d = embeddings_2d[n_train+n_val:]
    
    # Plot
    plt.figure(figsize=(12, 8))
    
    # Plot training points
    plt.scatter(train_2d[:, 0], train_2d[:, 1], c=train_labels, marker='o', label='Train', alpha=0.6)
    # Plot validation points
    plt.scatter(val_2d[:, 0], val_2d[:, 1], c=val_labels, marker='^', label='Validation', alpha=0.6)
    # Plot test points
    plt.scatter(test_2d[:, 0], test_2d[:, 1], c=test_labels, marker='s', label='Test', alpha=0.6)
    
    plt.colorbar(label='Class')
    plt.legend()
    plt.title('t-SNE Visualization of Embeddings')
    plt.show()

# Usage:
visualize_embeddings(train_embeddings, train_labels, val_embeddings, val_labels, test_embeddings, test_labels)

C:\Users\Mey\AppData\Local\Temp\ipykernel_7660\1301573674.py:111: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  siamese_model = torch.load("C:/Users/Mey/Documents/PlantDisea

KeyboardInterrupt: 